In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
from sklearn.metrics import (
    make_scorer,
    mean_absolute_error,
    mean_squared_error,
    r2_score
)
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.tree import DecisionTreeRegressor
import numpy as np

In [3]:
def root_mean_squared_error(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

METRICS_REGRESSION = {
    "MAE": make_scorer(mean_absolute_error),
    "MSE": make_scorer(mean_squared_error),
    "RMSE": make_scorer(root_mean_squared_error),
    "R2": make_scorer(r2_score)
}

In [4]:
df = pd.read_csv("../cenario1_engine.csv")

In [5]:
df.head()

,Brand_Mod,model,model_year,KM,fuel,HP,Litros,Cilindros,transmission_mod,ext_col_mod,int_col_mod,accident_mod,clean_title_mod,price_eur
0,Outra,Wrangler Sport,2014,114263,Gasoline,285.0,3.6,6.0,Automatic,Outra,Black,None reported,Yes,20020
1,Outra,Highlander XLE,2015,110361,Gasoline,270.0,3.5,6.0,Automatic,Outra,Outra,At least 1 accident or damage reported,Yes,23660
2,Outra,Camry Hybrid XLE,2018,112076,Hybrid,208.0,2.5,4.0,CVT,white,Black,At least 1 accident or damage reported,Yes,21835
3,Outra,Outback Touring XT,2021,53913,Gasoline,260.0,2.4,4.0,CVT,Outra,Outra,None reported,Yes,30940
4,Outra,GX 460 Base,2018,120701,Gasoline,301.0,4.6,8.0,Automatic,white,beige,At least 1 accident or damage reported,Yes,34666


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3207 entries, 0 to 3206
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Brand_Mod         3207 non-null   object 
 1   model             3207 non-null   object 
 2   model_year        3207 non-null   int64  
 3   KM                3207 non-null   int64  
 4   fuel              3207 non-null   object 
 5   HP                2578 non-null   float64
 6   Litros            2891 non-null   float64
 7   Cilindros         2705 non-null   float64
 8   transmission_mod  3207 non-null   object 
 9   ext_col_mod       3207 non-null   object 
 10  int_col_mod       3207 non-null   object 
 11  accident_mod      3207 non-null   object 
 12  clean_title_mod   3207 non-null   object 
 13  price_eur         3207 non-null   int64  
dtypes: float64(3), int64(3), object(8)
memory usage: 350.9+ KB


In [7]:
categorical_columns = df.select_dtypes(include='object').columns

# Mapeia as categorias para números
for column in categorical_columns:
    df[column] = df[column].astype('category').cat.codes

In [8]:
df.head()

,Brand_Mod,model,model_year,KM,fuel,HP,Litros,Cilindros,transmission_mod,ext_col_mod,int_col_mod,accident_mod,clean_title_mod,price_eur
0,4,1560,2014,114263,2,285.0,3.6,6.0,0,1,0,1,1,20020
1,4,771,2015,110361,2,270.0,3.5,6.0,0,1,1,0,1,23660
2,4,356,2018,112076,3,208.0,2.5,4.0,1,2,0,0,1,21835
3,4,1040,2021,53913,2,260.0,2.4,4.0,1,1,1,1,1,30940
4,4,700,2018,120701,2,301.0,4.6,8.0,0,2,2,0,1,34666


In [9]:
X, y = df.drop("price_eur", axis=1), df["price_eur"]

In [10]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=1234)

In [11]:
dt = DecisionTreeRegressor(max_depth=3, random_state=1234)
scores = cross_validate(dt, X, y, cv=splitter, scoring=METRICS_REGRESSION)
dt_scores = pd.DataFrame(scores)
pd.DataFrame(dt_scores.mean()).T

,fit_time,score_time,test_MAE,test_MSE,test_RMSE,test_R2
0,0.007076,0.0025,20430.530394,5.321180e+09,61419.87288,0.020798
